In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
import numpy as np

In [3]:
# TODO
# Hacer que prediga a precios constantes y después ajustar según la fecha

In [7]:
df = pd.read_csv('data/train.csv', index_col=['id'], parse_dates=['fecha'], error_bad_lines=False)

# Cargo la serie de inflación.
inflacion_mex = pd.read_csv('./data/inflacion_mex.csv',names=['fecha_mes','indice'],parse_dates=['fecha_mes'],dtype={'indice':np.float64})

# Cambio de base al ultimo dato.
infla2019 = inflacion_mex['indice'].tail(1).values[0]
inflacion_mex['indice'] = (inflacion_mex['indice']/infla2019)*100

# Elimino el número de día para poder joinear ambos DF.
inflacion_mex['fecha_mes'] = inflacion_mex['fecha_mes'].dt.to_period('M')
df['fecha_mes']= df['fecha'].dt.to_period('M')
df = df.merge(inflacion_mex,how='inner',on='fecha_mes')
df['precio_constante'] = (df['precio']/df['indice'])*100

In [8]:
subset = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'gimnasio', 'usosmultiples', 'piscina']
df = df.dropna(subset=subset)

In [9]:
df.columns

Index(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'fecha_mes', 'indice',
       'precio_constante'],
      dtype='object')

In [21]:
#create a dataframe with all training data except the target column
X = df[subset]
y = df['precio'].values

In [22]:
#split dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [12]:
# Create KNN regressor
#knn = KNeighborsRegressor(n_neighbors = 3)# Fit the classifier to the data
#knn.fit(X_train,y_train)

In [13]:
#show first 5 model predictions on the test data
#predicted = knn.predict(X_test)
#predicted[0:5]

In [14]:
#check MAE of our model on the test data
#mean_absolute_error(predicted, y_test)

In [15]:
#create a new KNN model
#knn_cv = KNeighborsRegressor(n_neighbors=3) #train model with cv of 5 
#cv_scores = cross_val_score(knn_cv, X, y, cv=5, scoring='neg_mean_absolute_error') #print each cv score (accuracy) and average them
#print(cv_scores)
#print('cv_scores mean:{}'.format(np.mean(cv_scores)))

In [16]:
#knn2.get_params().keys()

In [23]:
#create new a knn model
knn2 = KNeighborsRegressor() #create a dictionary of all values we want to test for n_neighbors
distances = ['euclidean', 'manhattan', 'chebyshev', 'minkowski']#, 'seuclidean']#, 'mahalanobis']#'wminkowski']#, ]
param_grid = {'n_neighbors': np.arange(3, 22, 2),
              'metric': distances} #use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(knn2, param_grid, scoring='neg_mean_absolute_error', cv=10) #fit model to data
knn_gscv.fit(X, y)

GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([ 3,  5,  7,  9, 11, 13, 15, 17, 19, 21]), 'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [24]:
#check top performing n_neighbors value
knn_gscv.best_params_

{'metric': 'manhattan', 'n_neighbors': 21}

In [25]:
#check mean score for the top performing value of n_neighbors
knn_gscv.best_score_

-765170.4611927088

In [ ]:
# error precios normales: 810808
# error precios constantes: -893153

In [29]:
test_df = pd.read_csv('data/test.csv', index_col=['id'], parse_dates=['fecha'], error_bad_lines=False)
test_df.fillna(0, inplace=True)
test_X = test_df[subset]

# tiene nas?

In [30]:
predicted = knn_gscv.predict(test_X)
predicted

array([2936952.38095238,  882683.33333333, 1661585.        , ...,
       1386547.61904762, 1141809.52380952, 2342380.95238095])

In [31]:
answer = pd.DataFrame(list(zip(test_df.index, predicted)), columns =['id', 'predicted']) 

In [32]:
answer

,id,predicted
0,4941,2.936952e+06
1,51775,8.826833e+05
2,115253,1.661585e+06
3,299321,2.251259e+06
4,173570,9.005396e+05
5,30862,1.798795e+06
6,244471,3.283048e+06
7,127794,1.095671e+06
8,71558,2.904286e+06
9,218011,1.824650e+06
